# Simulated transactions analysis - big data project

## Main jobs:
<ol>
  <li>obtain metrics by (expense_type, year) and (spending_class, year)</li>
  <li>calculate churn rate, cvv (customer loyalty), and other analytics</li>
</ol>

In [8]:
import org.apache.spark

import org.apache.spark


In [9]:
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner


In [10]:
import java.time.LocalDate
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit

import java.time.LocalDate
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit


## Case classes

After making imports, Transaction and CustomerSpending case classes are istanciated, in order to mantain a more readable code. It can be mapped

In [11]:
// case class definition for data clarity

case class Transaction(
  custId: String,
  startDate: String,
  endDate: String,
  transId: String,
  date: LocalDate,
  year: Int,
  month: Int,
  day: Int,
  expType: String,
  amount: Double
)

case class CustomerSpending(totalSpend: Double, spendingClass: String)

defined class Transaction
defined class CustomerSpending


In [12]:
// rdd mapping with created custom case classes 

val rdd = sc.textFile("sample_data_big_data.csv")
val header = rdd.first()
val dataRdd = rdd.filter(row => row != header)


val transactionsRdd = dataRdd.map { line =>
    val fields = line.split(",")
    val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")
    Transaction(
        custId   = fields(0),
        startDate = fields(1),
        endDate   = fields(2),
        transId   = fields(3),
        date      = LocalDate.parse(fields(4), formatter),
        year      = fields(5).toInt,
        month     = fields(6).toInt,
        day       = fields(7).toInt,
        expType   = fields(8),
        amount    = fields(9).toDouble
    )
}

rdd: org.apache.spark.rdd.RDD[String] = sample_data_big_data.csv MapPartitionsRDD[1] at textFile at <console>:42
header: String = CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
dataRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:44
transactionsRdd: org.apache.spark.rdd.RDD[Transaction] = MapPartitionsRDD[3] at map at <console>:47


In [13]:
// total spendings per customer
val customerSpendingsRdd = transactionsRdd
  .map(t => (t.custId, t.amount))
  .reduceByKey(_ + _)

// customer spendings classification
val classifiedSpendingsRdd = customerSpendingsRdd.map { case (custId, totalSpend) =>
  val spendingClass = if (totalSpend < 1000.0) "Poor"
                      else if (totalSpend < 10000.0) "Middle"
                      else "Rich"
  (custId, CustomerSpending(totalSpend, spendingClass))
}

// inner join transactions with classified spendings
val transactionsJoinedRdd = transactionsRdd
  .map(t => (t.custId, t))
  .join(classifiedSpendingsRdd)  // (custId, (Transaction, CustomerSpending))
  .map { case (custId, (transaction, custSpending)) =>
    (transaction.custId, transaction.transId, custSpending.spendingClass, transaction.year, transaction.expType, transaction.amount)
  }

customerSpendingsRdd: org.apache.spark.rdd.RDD[(String, Double)] = ShuffledRDD[5] at reduceByKey at <console>:43
classifiedSpendingsRdd: org.apache.spark.rdd.RDD[(String, CustomerSpending)] = MapPartitionsRDD[6] at map at <console>:46
transactionsJoinedRdd: org.apache.spark.rdd.RDD[(String, String, String, Int, String, Double)] = MapPartitionsRDD[11] at map at <console>:57


In [14]:
// criteria: (spending class, year)

// Mapping: ((spendingClass, year), (amount, Set(customerId)))
val aggregatedByCategoryYear = transactionsJoinedRdd
  .map { case (custId, transId, spendingClass, year, expType, amount) =>
    ((spendingClass, year), (amount, Set(custId)))
  }
  .reduceByKey { case ((amount1, custSet1), (amount2, custSet2)) =>
    (amount1 + amount2, custSet1 ++ custSet2)
  }

val metricsByCategoryYear = aggregatedByCategoryYear.map { case ((spendingClass, year), (totalAmount, custSet)) =>
  val customerCount = custSet.size
  val avgSpend = totalAmount / customerCount
  ((spendingClass, year), (totalAmount, customerCount, avgSpend))
}

aggregatedByCategoryYear: org.apache.spark.rdd.RDD[((String, Int), (Double, scala.collection.immutable.Set[String]))] = ShuffledRDD[13] at reduceByKey at <console>:45
metricsByCategoryYear: org.apache.spark.rdd.RDD[((String, Int), (Double, Int, Double))] = MapPartitionsRDD[14] at map at <console>:49


In [15]:
// criteria (expense type (exp), amount)

val aggregateByExpenseType = transactionsJoinedRdd
    .map { case (custId, transId, spendingClass, year, expType, amount) => 
        ((expType, year), (amount, Set(custId)))
    }
    .reduceByKey { case ((amount1, custSet1), (amount2, custSet2)) =>
        (amount1 + amount2, custSet1 ++ custSet2)
    }

val metricsByExpenseType = aggregateByExpenseType.map { case ((expType, year), (totalAmount, custSet)) =>
  val customerCount = custSet.size
  val avgSpend = totalAmount / customerCount
  ((expType, year), (totalAmount, customerCount, avgSpend))
}

metricsByExpenseType.collect()

aggregateByExpenseType: org.apache.spark.rdd.RDD[((String, Int), (Double, scala.collection.immutable.Set[String]))] = ShuffledRDD[16] at reduceByKey at <console>:44
metricsByExpenseType: org.apache.spark.rdd.RDD[((String, Int), (Double, Int, Double))] = MapPartitionsRDD[17] at map at <console>:48
res1: Array[((String, Int), (Double, Int, Double))] = Array(((Housing,2018),(74269.59,40,1856.73975)), ((Entertainment,2016),(83579.70999999999,3162,26.432545857052496)), ((Bills and Utilities,2016),(34274.36,178,192.55258426966293)), ((Motor/Travel,2013),(32845.76999999998,443,74.1439503386004)), ((Education,2017),(19444.319999999996,81,240.05333333333328)), ((Tax,2017),(30339.549999999996,59,514.2296610169491)), ((Entertainment,2015),(72523.83000000002,2817,25.74505857294995)), ((Bills and ...


# CVC

This section shows how you can manually extract features and informations from a dataset in order to create the CVC formula. This formula helps us to determine the customer value, and it helps deciding if we want to keep him.

In [17]:
val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")

formatter: java.time.format.DateTimeFormatter = Value(YearOfEra,4,19,EXCEEDS_PAD)'-'Value(MonthOfYear,2)'-'Value(DayOfMonth,2)


In [26]:
val first_and_last_transaction_rdd = transactionsRdd
    .map(t => (t.custId, (t.date, t.date)))
    .reduceByKey { case ((first1, last1), (first2, last2)) => 
        (if (first1.isBefore(first2)) first1 else first2, if (last1.isAfter(last2)) last1 else last2)
    }

first_and_last_transaction_rdd: org.apache.spark.rdd.RDD[(String, (java.time.LocalDate, java.time.LocalDate))] = ShuffledRDD[33] at reduceByKey at <console>:42


In [27]:
val frequencyRdd = transactionsRdd
    .map(t => (t.custId, 1))
    .reduceByKey(_ + _)

frequencyRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[35] at reduceByKey at <console>:42


In [20]:
import java.time.temporal.ChronoUnit;

val clvFeaturesRdd = first_and_last_transaction_rdd
    .join(frequencyRdd)
    .join(customerSpendingsRdd)
    .map { case (custId, (((firstDate, lastDate), freq), totalSpend)) =>  
        val lifetimeDays = ChronoUnit.DAYS.between(firstDate, lastDate) + 1
        val avgSpendingPerTxn = totalSpend / freq
        val estimatedLifetimeYears = lifetimeDays / 365.0
        val clv = (avgSpendingPerTxn * freq) * estimatedLifetimeYears  // Formula CLV base
        
        (custId, firstDate, lastDate, freq, totalSpend, avgSpendingPerTxn, estimatedLifetimeYears, clv)
    }

import java.time.temporal.ChronoUnit
clvFeaturesRdd: org.apache.spark.rdd.RDD[(String, java.time.LocalDate, java.time.LocalDate, Int, Double, Double, Double, Double)] = MapPartitionsRDD[28] at map at <console>:45
res4: Array[(String, java.time.LocalDate, java.time.LocalDate, Int, Double, Double, Double, Double)] = Array((CI0BAX5OMV,2019-07-25,2019-07-25,1,4.13,4.13,0.0027397260273972603,0.011315068493150685))


In [21]:
val customerSegmentsRdd = clvFeaturesRdd.map { case (custId, firstDate, lastDate, freq, totalSpend, avgSpendingPerTxn, estimatedLifetimeYears, clv) =>
  val segment = if (clv < 1000) "Low Value"
                else if (clv < 5000) "Medium Value"
                else "High Value"
  (custId, segment, clv)
}

customerSegmentsRdd: org.apache.spark.rdd.RDD[(String, String, Double)] = MapPartitionsRDD[29] at map at <console>:40


In [22]:
val loyaltyCategoryRdd = clvFeaturesRdd.map { case (custId, _, _, _, _, _, estimatedLifetimeYears, _) =>
  val loyaltyCategory = if (estimatedLifetimeYears < 1) "New Customer"
                        else if (estimatedLifetimeYears < 3) "Regular Customer"
                        else "Loyal Customer"
  (custId, loyaltyCategory, estimatedLifetimeYears)
}

loyaltyCategoryRdd: org.apache.spark.rdd.RDD[(String, String, Double)] = MapPartitionsRDD[30] at map at <console>:40


In [23]:
val churnRiskRdd = clvFeaturesRdd.map { case (custId, _, lastDate, _, _, _, _, clv) =>
  val today = LocalDate.now()
  val daysSinceLastPurchase = ChronoUnit.DAYS.between(lastDate, today)

  val churnRisk = if (clv < 1000 && daysSinceLastPurchase > 180) "High Risk"
                  else if (daysSinceLastPurchase > 90) "Medium Risk"
                  else "Low Risk"

  (custId, churnRisk, daysSinceLastPurchase, clv)
}

churnRiskRdd: org.apache.spark.rdd.RDD[(String, String, Long, Double)] = MapPartitionsRDD[31] at map at <console>:40
